In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 28.0 MB/s eta 0:00:0

In [2]:
import pandas as pd
from simpletransformers.t5 import T5Model

In [3]:
## Read data
train_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_train.csv").astype(str)[['prefix','input_text','target_text']]
eval_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_val.csv").astype(str)[['prefix','input_text','target_text']]
test_df = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_test.csv").astype(str)[['prefix','input_text','target_text']]

## Select data-points with prefix = 'keywords'
train_df = train_df[train_df['prefix']=='keywords'].copy()
eval_df = eval_df[eval_df['prefix']=='keywords'].copy()
test_df = test_df[test_df['prefix']=='keywords'].copy()

train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

print("Train shape: ", train_df.shape)
print("Val shape: ", eval_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (14072, 3)
Val shape:  (1787, 3)
Test shape:  (1761, 3)


In [4]:
## Output

train_keywords_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-keywords/df_train_pred_kw.csv"
val_keywords_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-keywords/df_val_pred_kw.csv"
test_keywords_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-keywords/df_test_pred_kw.csv"

model_op_path = "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-keywords/"


In [5]:
model_args = {
    "max_seq_length": 250,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "num_train_epochs": 5,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "fp16": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "learning_rate":1e-4,
    "weight_decay":0.01
}
model = T5Model("t5", "t5-base", args=model_args)


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
model.train_model(train_df, eval_data=eval_df, output_dir=model_op_path)


  0%|          | 0/14072 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/880 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Running Epoch 4 of 5:   0%|          | 0/880 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

(4400,
 {'global_step': [880, 1760, 2640, 3520, 4400],
  'eval_loss': [0.7310500754309552,
   0.6811346842774323,
   0.6918423803789275,
   0.7070800478437117,
   0.7272228545376233],
  'train_loss': [0.37253281474113464,
   0.20404505729675293,
   0.5917540192604065,
   0.2908511459827423,
   0.22363516688346863]})

In [5]:
model_args_test = {
    "overwrite_output_dir": True,
    "max_seq_length": 250,
    "eval_batch_size": 32,
    "use_multiprocessing": False,
    "num_beams": 3,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

# Load the trained model
model = T5Model("t5", model_op_path + "outputs/best_model", args=model_args_test)

In [6]:
# Predict explanations function

def predict_explanations(df):
  # Prepare the data for predictions
  to_predict = [
      prefix + ": " + str(input_text)
      for prefix, input_text in zip(df["prefix"].tolist(), df["input_text"].tolist())
  ]
  truth = df["target_text"].tolist()
  tasks = df["prefix"].tolist()

  # Get the model predictions
  preds = model.predict(to_predict)

  ## Taking only the top (first) predictions
  top_preds = [pred[0] for pred in preds]
  df["predicted"] = top_preds
  df['all_preds'] = preds


In [7]:
## Predict on train, val and test -  chance of overfitting

## Uncomment the following for predictions
print("Predicting train")
predict_explanations(train_df)
train_df.to_csv(train_keywords_path, index=False)

print("Predicting val")
predict_explanations(eval_df)
eval_df.to_csv(val_keywords_path, index=False)

print("Predicting test")
predict_explanations(test_df)
test_df.to_csv(test_keywords_path, index=False)


Predicting train


Generating outputs:   0%|          | 0/440 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3745: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/42216 [00:00<?, ?it/s]

Predicting val


...

Decoding outputs:   0%|          | 0/5361 [00:00<?, ?it/s]

Predicting test


Generating outputs:   0%|          | 0/56 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3745: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/5283 [00:00<?, ?it/s]